# Debugging `npm run setup` and Docker Build Issues

This notebook provides guidance on understanding and troubleshooting issues with the `npm run setup` command and related Docker build processes in the QI v2 LLM project.

## Overview of `npm run setup`

The `npm run setup` command is a crucial step in initializing the development environment for the QI v2 LLM project. This command performs several important tasks:

- Installs required dependencies
- Builds Docker containers
- Configures the development environment
- Prepares the project for development work

The setup process is defined in the `package.json` file, typically running a series of scripts that automate the environment setup process.

In [ ]:
# Example of how to run the setup command
npm run setup

### What happens during setup?

When you run `npm run setup`, the following typically occurs:

1. Dependencies are checked and installed
2. Docker images are built using Dockerfiles
3. Configuration files may be generated
4. Environment variables may be set
5. Initial database migrations or seed data may be created

Understanding this process helps when troubleshooting setup issues.

## Understanding the Docker Build Process

Docker builds in this project use a multi-stage approach to create optimized containers. Let's explore the key components of this process:

### Dockerfiles

Dockerfiles define the steps needed to create a Docker image. They specify:
- Base images
- Dependencies to install
- Files to copy into the container
- Commands to run
- Environment variables
- Exposed ports

### Multi-stage Builds

Multi-stage builds allow the use of multiple FROM statements in a Dockerfile. Each FROM instruction starts a new stage of the build. This helps to:
- Keep final images small
- Separate build-time dependencies from runtime dependencies
- Organize the build process into logical stages

In [ ]:
# Command to view Docker images
docker images

# Command to view Docker build history
docker history image_name

### Inspecting the Dockerfile

Let's look at a simplified version of what our Dockerfile might contain:

In [ ]:
# Display the contents of the Dockerfile
import os

dockerfile_path = "/home/zzhang/dev/qi/github/qi-v2-llm/Dockerfile"
if os.path.exists(dockerfile_path):
    with open(dockerfile_path, 'r') as file:
        print(file.read())
else:
    print("Dockerfile not found at the expected location.")

## Analyzing the Error Logs

When Docker builds fail, the error logs provide crucial information to diagnose the issue. Common problems include:

1. Network timeouts during package downloads
2. Missing dependencies
3. Incompatible versions of packages
4. Resource limitations (memory, disk space)
5. Permission issues

Let's look at how to analyze these logs effectively:

In [ ]:
# Function to parse Docker build logs and identify common errors
def analyze_docker_logs(log_content):
    common_issues = {
        'network': ['network error', 'timeout', 'connection refused', 'could not resolve'],
        'dependency': ['not found', 'no such package', 'missing dependency'],
        'version': ['version conflict', 'incompatible', 'requires version'],
        'resource': ['no space left', 'out of memory', 'killed'],
        'permission': ['permission denied', 'access denied']
    }
    
    issues_found = {category: [] for category in common_issues}
    
    for line in log_content.split('\n'):
        for category, keywords in common_issues.items():
            for keyword in keywords:
                if keyword.lower() in line.lower():
                    issues_found[category].append(line.strip())
                    break
    
    return issues_found

# Example usage
sample_log = """
#6 [internal] load build definition from Dockerfile
#6 transferring dockerfile: 36B done
#7 ERROR: failed to solve: process "/bin/sh -c pip install -r requirements.txt" did not complete successfully: exit code: 1
#7   398.9 ERROR: Could not install packages due to an OSError: [Errno 104] Connection reset by peer
#7   399.0     It appears you don't have a complete pip download cache directory locally or pip's cache is corrupted.
#7   399.0         You need to fix pip's cache or run pip with --no-cache-dir.
"""

analysis = analyze_docker_logs(sample_log)
for category, lines in analysis.items():
    if lines:
        print(f"\n{category.upper()} ISSUES:")
        for line in lines:
            print(f"  - {line}")

### Common Error Patterns

1. **PIP Download Errors**:
   ```
   ERROR: Could not install packages due to an OSError: [Errno 104] Connection reset by peer
   ```
   This indicates network instability during package download.

2. **Package Build Failures**:
   ```
   error: command 'gcc' failed with exit status 1
   ```
   This suggests missing build dependencies for a Python package.

3. **Memory Issues**:
   ```
   Killed
   ```
   A simple "Killed" message often means the process ran out of memory.

4. **Disk Space Issues**:
   ```
   no space left on device
   ```
   This clearly indicates running out of disk space during the build.

## Retrying the Installation with Resumption

When package installation fails due to network issues, using pip's `--resume-retries` option can help resume interrupted downloads. This is particularly useful for large packages or unstable connections.

### Steps to Retry Installation

In [ ]:
# Example of using the --resume-retries option with pip
pip install -r requirements.txt --resume-retries 10

### Modifying the Dockerfile

To incorporate better retry handling in your Docker builds, you can modify the Dockerfile to use these options:

In [ ]:
# Example of a Dockerfile snippet that incorporates retry mechanisms
dockerfile_snippet = """
# Use a base image
FROM python:3.9

# Set working directory
WORKDIR /app

# Copy requirements file
COPY requirements.txt .

# Install Python dependencies with retry mechanism
RUN pip install --no-cache-dir --retries 5 --timeout 100 -r requirements.txt || \\
    pip install --no-cache-dir --retries 10 --timeout 200 -r requirements.txt

# Copy application code
COPY . .

# Command to run the application
CMD ["python", "app.py"]
"""

print(dockerfile_snippet)

### Other Strategies for Handling Installation Issues

1. **Network Timeout Settings**: Increase timeout values for package managers
2. **Mirror Repositories**: Use local or more reliable mirrors for package downloads
3. **Progressive Installation**: Install packages in smaller batches
4. **Caching**: Implement proper caching in multi-stage Docker builds
5. **Pre-downloading Dependencies**: Download dependencies before building the Docker image

In [ ]:
# Example of using a mirror repository with pip
pip install -r requirements.txt --index-url https://pypi.tuna.tsinghua.edu.cn/simple

# Example of progressive installation
cat requirements.txt | xargs -n 1 pip install

## Region-Specific Setup Configuration

This project includes specialized configurations for users in regions with network restrictions. The repository contains scripts to switch between global and China-specific modes for both Docker and GitHub configurations.

### China-Specific Configuration

If you're experiencing connectivity issues in China, you can use the provided scripts to switch to mirrors and configurations optimized for the region:

In [ ]:
# Switch Docker to China mode (uses Chinese mirrors for faster access)
npm run docker:china-mode

# Switch GitHub to China mode
npm run github:china-mode

These commands execute the scripts in the `scripts/` directory that make the necessary configuration changes. The changes typically include:

1. Setting Docker to use Chinese mirrors such as the official Docker China registry
2. Configuring npm to use Chinese mirrors like Taobao npm registry
3. Setting Python's pip to use mirrors like Tsinghua University's PyPI mirror
4. Adjusting Git configuration for better performance in the region

To revert back to global settings:

In [ ]:
# Switch Docker back to global mode
npm run docker:global-mode

# Switch GitHub back to global mode
npm run github:global-mode

## DeepSeek API Configuration Troubleshooting

The QI v2 LLM project integrates with DeepSeek AI models for enhanced AI capabilities. Issues with this integration are typically related to API keys, network connectivity, or configuration problems.

### Common DeepSeek API Issues

1. **Missing or Invalid API Key**:
   - Error message: `Authentication error: Invalid API key provided`
   - Solution: Re-run the `npm run secrets:setup-deepseek` command and provide a valid API key

2. **Network Connectivity to DeepSeek API**:
   - Error message: `Connection refused` or `Timeout connecting to api.deepseek.com`
   - Solution: Check internet connectivity or configure appropriate proxies

3. **Authentication Method Issues**:
   - Error message: `Unsupported authentication method`
   - Solution: Ensure you're using the correct authentication headers as specified in DeepSeek documentation

4. **Rate Limiting**:
   - Error message: `Too many requests` or `Rate limit exceeded`
   - Solution: Implement backoff strategies or reduce the frequency of API calls

In [ ]:
# Test DeepSeek API configuration
npm run deepseek:test

If the test fails, you may need to manually verify your API key and configuration:

In [ ]:
# Script to verify DeepSeek API configuration
import os
import requests

def test_deepseek_api():
    api_key = os.environ.get('DEEPSEEK_API_KEY')
    
    if not api_key:
        print("Error: DEEPSEEK_API_KEY environment variable not set")
        print("Run 'npm run secrets:setup-deepseek' to configure it")
        return False
        
    # Simple test request to DeepSeek API
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "deepseek-coder",
        "messages": [{"role": "user", "content": "Say hello"}],
        "max_tokens": 10
    }
    
    try:
        response = requests.post("https://api.deepseek.com/v1/chat/completions", 
                              headers=headers, json=data, timeout=10)
        response.raise_for_status()
        print("DeepSeek API test successful")
        print(f"Response: {response.json()}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error testing DeepSeek API: {e}")
        return False

# Run the test function
test_deepseek_api()

## NPM-Specific Troubleshooting

While Docker issues are common during setup, you may also encounter NPM-related problems. Here are some typical issues and their solutions:

### Common NPM Issues

1. **Missing Dependencies**:
   ```
   Error: Cannot find module 'module-name'
   ```
   Solution: Run `npm install` to install dependencies

2. **Permission Errors**:
   ```
   EACCES: permission denied
   ```
   Solution: Use `sudo npm install` or fix permissions with `npm config set prefix ~/npm`

3. **Outdated NPM**:
   ```
   npm WARN npm npm does not support Node.js v14.17.0
   ```
   Solution: Update npm with `npm install -g npm@latest`

4. **Package Lock Conflicts**:
   ```
   npm ERR! Conflicting peer dependency
   ```
   Solution: Remove package-lock.json and node_modules folder, then run `npm install`

5. **Network Errors**:
   ```
   npm ERR! network timeout
   ```
   Solution: Use a mirror with `npm config set registry https://registry.npmmirror.com/`

In [ ]:
# Clean npm cache and reinstall
npm cache clean --force
rm -rf node_modules package-lock.json
npm install

### Diagnosing NPM Script Issues

To debug a specific NPM script from package.json, you can run it with the `--verbose` flag:

In [ ]:
# Run a specific npm script with verbose output
npm run setup --verbose

To inspect what commands are being run by a specific npm script without executing them:

In [ ]:
// Node.js script to analyze package.json scripts
const fs = require('fs');
const path = require('path');

// Read package.json
const packageJsonPath = path.resolve('/home/zzhang/dev/qi/github/qi-v2-llm/package.json');
try {
    const packageJson = JSON.parse(fs.readFileSync(packageJsonPath, 'utf8'));
    
    // Display all scripts
    console.log('Available npm scripts:\n');
    Object.entries(packageJson.scripts || {}).forEach(([name, command]) => {
        console.log(`${name}:\n  ${command}\n`);
    });
    
    // Check for setup script
    if (packageJson.scripts && packageJson.scripts.setup) {
        console.log('\nSetup script breakdown:\n');
        
        // Simple parsing of the setup script
        const setupCommands = packageJson.scripts.setup.split('&&').map(cmd => cmd.trim());
        setupCommands.forEach((cmd, index) => {
            console.log(`${index + 1}. ${cmd}`);
            
            // Check if this command runs another npm script
            const npmRunMatch = cmd.match(/npm run ([\w\-:]+)/);
            if (npmRunMatch && npmRunMatch[1] && packageJson.scripts[npmRunMatch[1]]) {
                console.log(`   → Runs: ${packageJson.scripts[npmRunMatch[1]]}`);
            }
        });
    }
} catch (error) {
    console.error(`Error analyzing package.json: ${error.message}`);
}

## MCP Container Troubleshooting

The Model Context Protocol (MCP) container is the default development environment for this project. It combines Python and TypeScript tools with specialized components for LLM integration.

### Common MCP Container Issues

1. **Container Fails to Start**:
   ```
   Error response from daemon: driver failed programming external connectivity
   ```
   Solution: Check for port conflicts or restart Docker

2. **Missing Python Packages in MCP Environment**:
   ```
   ModuleNotFoundError: No module named 'package_name'
   ```
   Solution: Install the package inside the MCP container with `pip install package_name`

3. **TypeScript Compilation Errors**:
   ```
   error TS2307: Cannot find module or its corresponding type declarations
   ```
   Solution: Install the package and its types with `npm install package @types/package`

4. **Permission Issues with Workspace Files**:
   ```
   Error: EACCES: permission denied, open '/workspace/file.txt'
   ```
   Solution: Fix permissions with `sudo chown -R $(id -u):$(id -g) /workspace`

In [ ]:
# Check MCP container status
docker-compose ps mcp

# Restart the MCP container
npm run mcp:restart

# Access MCP container shell for debugging
npm run mcp

## Testing the Environment

After successfully completing the setup process, it's important to verify that the environment is correctly configured. This section demonstrates how to test various aspects of the development environment.

In [ ]:
# Check Docker container status
docker ps

# Check if the application containers are running
docker-compose ps

In [ ]:
# Simple Python script to verify the environment
import sys
import platform
import subprocess

def check_environment():
    results = {
        "Python Version": sys.version,
        "Platform": platform.platform(),
        "Docker Version": None,
        "Node Version": None,
        "NPM Version": None
    }
    
    try:
        results["Docker Version"] = subprocess.check_output(
            ["docker", "--version"], 
            stderr=subprocess.STDOUT
        ).decode('utf-8').strip()
    except:
        results["Docker Version"] = "Not installed or not accessible"
    
    try:
        results["Node Version"] = subprocess.check_output(
            ["node", "--version"], 
            stderr=subprocess.STDOUT
        ).decode('utf-8').strip()
    except:
        results["Node Version"] = "Not installed or not accessible"
    
    try:
        results["NPM Version"] = subprocess.check_output(
            ["npm", "--version"], 
            stderr=subprocess.STDOUT
        ).decode('utf-8').strip()
    except:
        results["NPM Version"] = "Not installed or not accessible"
    
    return results

env_check = check_environment()
for key, value in env_check.items():
    print(f"{key}: {value}")

### Verifying Application Functionality

After checking that the environment is set up correctly, you should verify that the application itself works as expected. This might involve:

1. Running test suites
2. Making API requests to services
3. Verifying database connections
4. Checking that UI components render correctly

Here's a simple example of testing an endpoint:

In [ ]:
import requests
import json

def test_api_endpoint(url):
    try:
        response = requests.get(url)
        print(f"Status Code: {response.status_code}")
        if response.status_code == 200:
            print("Response Content:")
            try:
                print(json.dumps(response.json(), indent=2))
            except:
                print(response.text[:500] + "..." if len(response.text) > 500 else response.text)
        else:
            print(f"Error: Received status code {response.status_code}")
    except Exception as e:
        print(f"Error connecting to {url}: {str(e)}")

# Example usage - replace with actual endpoint
# test_api_endpoint("http://localhost:3000/api/status")
print("Uncomment and update the example with an actual endpoint to test")

### Testing DeepSeek Integration

To verify that the DeepSeek API integration is working correctly:

In [ ]:
# Test DeepSeek API integration
import os
import requests

def test_deepseek_integration():
    api_key = os.environ.get('DEEPSEEK_API_KEY')
    
    if not api_key:
        print("DEEPSEEK_API_KEY not found in environment variables")
        return
    
    # Simple test prompt
    prompt = "Generate a simple TypeScript function that adds two numbers."
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "deepseek-coder",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 150
    }
    
    try:
        response = requests.post(
            "https://api.deepseek.com/v1/chat/completions",
            headers=headers,
            json=data
        )
        
        if response.status_code == 200:
            result = response.json()
            print("DeepSeek API test successful!\n")
            print("Generated code:")
            print(result['choices'][0]['message']['content'])
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
    except Exception as e:
        print(f"Exception: {str(e)}")

# Uncomment to run the test
# test_deepseek_integration()

## Conclusion

In this notebook, we've covered:

1. The purpose and process of `npm run setup`
2. Understanding Docker build processes and multi-stage builds
3. Techniques for analyzing and diagnosing build errors
4. Methods to handle network issues during package installation
5. Region-specific configuration for users in China
6. DeepSeek API integration troubleshooting
7. NPM-specific troubleshooting techniques
8. MCP container specific issues and solutions
9. How to verify that the environment is correctly set up

By understanding these aspects, you'll be better equipped to troubleshoot and resolve issues that may arise during the setup process for the QI v2 LLM project.